In [1]:
#!/usr/bin/env python3
"""
llm_performance_analysis.py

Analyzes performance across LLMs while preserving your 5-method methodology.

Outputs:
1) TABLE L1: LLM_for_Workflow × Method (All runs)
2) TABLE L2: Pass-rate pivot (LLM rows × Method columns)
3) TABLE L3: ORT pivot (LLM rows × Method columns)
4) TABLE L4 (optional): LLM_for_Workflow × Orchestrator (aggregated over methods)
5) TABLE L5 (optional): LLM_for_Workflow × Orchestrator × Method (big; appendix only)

Notes:
- Uses Std_LLM for Direct (Non-Reasoning) + Prompt2DAG strategies
- Uses Reasoning_LLM for Direct (Reasoning)
- Computes ORT_scaled using the same approach as your main analysis (global min-max scaling)
"""

import pandas as pd
import numpy as np

# -----------------------
# Config
# -----------------------
CSV_PATH = "/Users/abubakarialidu/Desktop/Data Result/all_sessions_cleaned.csv"

STANDARD_LLMS = (
  "deepinfra:deepseek_ai",
  "deepinfra:meta_llama",
  "deepinfra:qwen",
  "deepinfra:qwen3",
  "deepinfra:microsoft_phi",
  "deepinfra:claude-4-sonnet",
  "deepinfra:mistralaiSmall",
)

REASONING_LLMS = (
  "deepinfra:Qwen3-235B-A22B-Thinking-2507",
  "deepinfra:gemini-2.5-pro",
)

ORCH_ORDER = ["airflow", "dagster", "prefect"]
METHOD_ORDER = [
    "Direct (Non-Reasoning)",
    "Prompt2DAG (Template)",
    "Prompt2DAG (LLM)",
    "Prompt2DAG (Hybrid)",
    "Direct (Reasoning)",
]

# ORT penalty weights (match your main analysis)
ALPHA_CRIT = 2.0
BETA_MAJOR = 1.0
GAMMA_MINOR = 0.25

pd.set_option("display.max_columns", 200)
pd.set_option("display.width", 240)

# -----------------------
# Helpers
# -----------------------
def normalize_llm_name(x: str) -> str:
    """
    Make LLM identifiers comparable across formats:
    - dataset often uses deepinfra-deepseek_ai, while your lists use deepinfra:deepseek_ai
    """
    if x is None:
        return ""
    x = str(x).strip()
    x = x.replace("deepinfra:", "deepinfra-")
    x = x.replace(":", "-")
    x = x.replace("/", "_")
    x = x.replace(" ", "")
    return x

STD_SET = set(normalize_llm_name(x) for x in STANDARD_LLMS)
RSN_SET = set(normalize_llm_name(x) for x in REASONING_LLMS)
ALL_LLM_SET = STD_SET.union(RSN_SET)

def classify_method(row):
    workflow = row.get("Workflow", "")
    strategy = str(row.get("Strategy") or "").lower()
    if workflow == "Direct":
        return "Direct (Non-Reasoning)"
    elif workflow == "Reasoning":
        return "Direct (Reasoning)"
    elif workflow == "Prompt2DAG":
        if "template" in strategy:
            return "Prompt2DAG (Template)"
        elif "llm" in strategy:
            return "Prompt2DAG (LLM)"
        elif "hybrid" in strategy:
            return "Prompt2DAG (Hybrid)"
        else:
            return f"Prompt2DAG ({row.get('Strategy','Unknown')})"
    else:
        return workflow

def mean_sd(x):
    x = pd.to_numeric(x, errors="coerce").dropna()
    if len(x) == 0:
        return "NA"
    return f"{x.mean():.2f} ± {x.std(ddof=1):.2f}"

# -----------------------
# Load
# -----------------------
df = pd.read_csv(CSV_PATH)
print(f"Loaded {len(df):,} rows, {len(df.columns)} cols")

# Normalize orchestrator
df["Orchestrator"] = df["Orchestrator"].astype(str).str.lower().str.strip()

# Passed → bool
if df["Passed"].dtype != bool:
    df["Passed"] = df["Passed"].astype(str).str.lower().map({"true": True, "false": False, "1": True, "0": False})

# Method
df["Method"] = df.apply(classify_method, axis=1)
df = df[df["Method"].isin(METHOD_ORDER)].copy()

# Ensure columns
for c in ["Std_LLM", "Reasoning_LLM"]:
    if c not in df.columns:
        df[c] = "unknown"

for c in ["Static_Score", "Compliance_Score", "Combined_Score"]:
    if c not in df.columns:
        raise ValueError(f"Missing required score column: {c}")

for c in ["Critical_Issues", "Major_Issues", "Minor_Issues"]:
    if c not in df.columns:
        df[c] = 0
    df[c] = df[c].fillna(0)

df["Total_Issues"] = df["Critical_Issues"] + df["Major_Issues"] + df["Minor_Issues"]

# LLM_for_Workflow (Std for Direct/P2D; Reasoning for Reasoning)
def get_llm_for_workflow(row):
    if row["Method"] == "Direct (Reasoning)":
        return normalize_llm_name(row.get("Reasoning_LLM", ""))
    else:
        return normalize_llm_name(row.get("Std_LLM", ""))

df["LLM_for_Workflow"] = df.apply(get_llm_for_workflow, axis=1)

# Filter to declared LLMs only (recommended for clean appendix tables)
df = df[df["LLM_for_Workflow"].isin(ALL_LLM_SET)].copy()
print(f"Rows after keeping only declared LLMs: {len(df):,}")

# -----------------------
# ORT_scaled (global min-max scaling like your main analysis)
# -----------------------
df["Base_Score"] = np.where(df["Passed"] == True, df["Combined_Score"], 0.0)
df["Penalty"] = (
    ALPHA_CRIT * df["Critical_Issues"] +
    BETA_MAJOR * df["Major_Issues"] +
    GAMMA_MINOR * df["Minor_Issues"]
)
df["ORT_raw"] = df["Base_Score"] - df["Penalty"]

ort_min = df["ORT_raw"].min()
ort_max = df["ORT_raw"].max()
df["ORT_scaled"] = 0.0
if ort_max > ort_min:
    df["ORT_scaled"] = 10.0 * (df["ORT_raw"] - ort_min) / (ort_max - ort_min)

# -----------------------
# TABLE L1: LLM × Method (All runs)
# -----------------------
grp = df.groupby(["LLM_for_Workflow", "Method"], observed=True)

t1 = grp.agg(
    N=("Passed", "size"),
    N_Passed=("Passed", "sum"),
    Pass_Rate=("Passed", "mean"),
    SAT_mean=("Static_Score", "mean"),
    SAT_sd=("Static_Score", "std"),
    PCT_mean=("Compliance_Score", "mean"),
    PCT_sd=("Compliance_Score", "std"),
    Combined_mean=("Combined_Score", "mean"),
    Combined_sd=("Combined_Score", "std"),
    ORT_mean=("ORT_scaled", "mean"),
    ORT_sd=("ORT_scaled", "std"),
    Crit=("Critical_Issues", "mean"),
    Major=("Major_Issues", "mean"),
    Minor=("Minor_Issues", "mean"),
    Issues=("Total_Issues", "mean"),
).reset_index()

t1["Pass%"] = (100 * t1["Pass_Rate"]).round(1)
t1["SAT"] = t1.apply(lambda r: f"{r['SAT_mean']:.2f} ± {r['SAT_sd']:.2f}", axis=1)
t1["PCT"] = t1.apply(lambda r: f"{r['PCT_mean']:.2f} ± {r['PCT_sd']:.2f}", axis=1)
t1["Combined"] = t1.apply(lambda r: f"{r['Combined_mean']:.2f} ± {r['Combined_sd']:.2f}", axis=1)
t1["ORT_scaled"] = t1.apply(lambda r: f"{r['ORT_mean']:.2f} ± {r['ORT_sd']:.2f}", axis=1)

t1_out = t1[[
    "LLM_for_Workflow", "Method", "N", "N_Passed", "Pass%", "SAT", "PCT", "Combined", "ORT_scaled",
    "Issues", "Crit", "Major", "Minor"
]].copy()

t1_out["Method"] = pd.Categorical(t1_out["Method"], METHOD_ORDER, ordered=True)
t1_out = t1_out.sort_values(["LLM_for_Workflow", "Method"]).reset_index(drop=True)

print("\n" + "="*120)
print("TABLE L1: LLM_for_Workflow × Method (All runs)  [appendix-ready]")
print("="*120)
print(t1_out.to_string(index=False))

# -----------------------
# TABLE L2: Pass rate pivot (LLM rows × Method cols)
# -----------------------
pass_pivot = (
    grp["Passed"].mean().reset_index()
    .assign(PassPct=lambda d: (100*d["Passed"]).round(1))
    .pivot(index="LLM_for_Workflow", columns="Method", values="PassPct")
)
pass_pivot = pass_pivot.reindex(columns=METHOD_ORDER)

print("\n" + "="*120)
print("TABLE L2: Pass rate (%) pivot  [LLM rows × Method cols]")
print("="*120)
print(pass_pivot.fillna("").to_string())

# -----------------------
# TABLE L3: ORT pivot (LLM rows × Method cols)
# -----------------------
ort_pivot = (
    grp["ORT_scaled"].mean().reset_index()
    .assign(ORT=lambda d: d["ORT_scaled"].round(2))
    .pivot(index="LLM_for_Workflow", columns="Method", values="ORT")
)
ort_pivot = ort_pivot.reindex(columns=METHOD_ORDER)

print("\n" + "="*120)
print("TABLE L3: ORT_scaled (mean) pivot  [LLM rows × Method cols]")
print("="*120)
print(ort_pivot.fillna("").to_string())

# -----------------------
# TABLE L4: LLM × Orchestrator (aggregated over methods)
# -----------------------
grp2 = df.groupby(["LLM_for_Workflow", "Orchestrator"], observed=True)

t4 = grp2.agg(
    N=("Passed", "size"),
    Pass_Rate=("Passed", "mean"),
    ORT=("ORT_scaled", "mean"),
    Combined=("Combined_Score", "mean")
).reset_index()

t4["Pass%"] = (100*t4["Pass_Rate"]).round(1)
t4["ORT"] = t4["ORT"].round(2)
t4["Combined"] = t4["Combined"].round(2)

t4["Orchestrator"] = pd.Categorical(t4["Orchestrator"], ORCH_ORDER, ordered=True)
t4 = t4.sort_values(["LLM_for_Workflow", "Orchestrator"]).reset_index(drop=True)

print("\n" + "="*120)
print("TABLE L4: LLM_for_Workflow × Orchestrator (aggregated over methods)  [optional appendix]")
print("="*120)
print(t4[["LLM_for_Workflow","Orchestrator","N","Pass%","ORT","Combined"]].to_string(index=False))

# -----------------------
# TABLE L5: LLM × Orchestrator × Method (BIG, but useful for appendix/diagnostics)
# -----------------------
grp3 = df.groupby(["LLM_for_Workflow", "Orchestrator", "Method"], observed=True)

t5 = grp3.agg(
    N=("Passed", "size"),
    Pass_Rate=("Passed", "mean"),
    ORT=("ORT_scaled", "mean"),
    Combined=("Combined_Score", "mean")
).reset_index()

t5["Pass%"] = (100*t5["Pass_Rate"]).round(1)
t5["ORT"] = t5["ORT"].round(2)
t5["Combined"] = t5["Combined"].round(2)

t5["Orchestrator"] = pd.Categorical(t5["Orchestrator"], ORCH_ORDER, ordered=True)
t5["Method"] = pd.Categorical(t5["Method"], METHOD_ORDER, ordered=True)
t5 = t5.sort_values(["LLM_for_Workflow","Orchestrator","Method"]).reset_index(drop=True)

print("\n" + "="*120)
print("TABLE L5: LLM_for_Workflow × Orchestrator × Method (compact metrics)  [big appendix table]")
print("="*120)
print(t5[["LLM_for_Workflow","Orchestrator","Method","N","Pass%","ORT","Combined"]].to_string(index=False))



Loaded 8,742 rows, 94 cols
Rows after keeping only declared LLMs: 8,742

TABLE L1: LLM_for_Workflow × Method (All runs)  [appendix-ready]
                       LLM_for_Workflow                 Method   N  N_Passed  Pass%         SAT         PCT    Combined  ORT_scaled   Issues     Crit    Major    Minor
deepinfra-Qwen3-235B-A22B-Thinking-2507     Direct (Reasoning) 342       243   71.1 5.85 ± 1.54 5.90 ± 2.34 5.94 ± 1.82 7.10 ± 1.76 6.061404 0.415205 1.649123 3.997076
              deepinfra-claude-4-sonnet Direct (Non-Reasoning) 342       168   49.1 4.97 ± 1.67 5.19 ± 1.98 5.13 ± 1.77 6.11 ± 1.94 7.263158 0.368421 1.976608 4.918129
              deepinfra-claude-4-sonnet  Prompt2DAG (Template) 249       138   55.4 4.25 ± 3.66 4.25 ± 3.85 4.29 ± 3.76 6.39 ± 1.52 5.176707 0.702811 2.216867 2.257028
              deepinfra-claude-4-sonnet       Prompt2DAG (LLM) 333       222   66.7 6.99 ± 0.41 4.97 ± 3.56 6.03 ± 1.92 6.89 ± 1.69 7.120120 0.414414 1.750751 4.954955
              deepinfr